### Imports

In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://Varun-CK:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1579455488715)
SparkSession available as 'spark'


2020-01-19 23:08:06 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.sql.SparkSession


### Creating Spark Session

In [2]:
val spark = SparkSession.builder().getOrCreate()

2020-01-19 23:08:29 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2a1822d5


### Register temp table

In [4]:
val dfTags = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("..\\Resources\\question_tags_10K.csv")
    .toDF("id", "tag")

dfTags: org.apache.spark.sql.DataFrame = [id: int, tag: string]


In [5]:
dfTags.createOrReplaceTempView("so_tags")

### List all tables in Spark's catalog

In [6]:
spark.catalog.listTables().show()

+-------+--------+-----------+---------+-----------+
|   name|database|description|tableType|isTemporary|
+-------+--------+-----------+---------+-----------+
|so_tags|    null|       null|TEMPORARY|       true|
+-------+--------+-----------+---------+-----------+



### List all tables in Spark's catalog using Spark SQL

In [7]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |  so_tags|       true|
+--------+---------+-----------+



### Select columns

In [10]:
// dfTags.select("id", "tag").limit(10).show()

In [12]:
spark.sql("select id, tag from so_tags limit 10").show(5)

+---+---------------+
| id|            tag|
+---+---------------+
|  1|           data|
|  4|             c#|
|  4|       winforms|
|  4|type-conversion|
|  4|        decimal|
+---+---------------+
only showing top 5 rows



### Filter by column value

In [13]:
// dfTags.filter("tag == 'php'").show(10)

In [14]:
spark.sql("select * from so_tags where tag = 'php'").show(5)

+---+---+
| id|tag|
+---+---+
| 23|php|
| 42|php|
| 85|php|
|126|php|
|146|php|
+---+---+
only showing top 5 rows



### Count number of rows

In [15]:
// println(s"Number of php tags = ${ dfTags.filter("tag == 'php'").count() }")

In [18]:
spark.sql("""select count(*) as php_count from so_tags where tag='php'""".stripMargin).show(5)

+---------+
|php_count|
+---------+
|      133|
+---------+



### SQL like

In [19]:
// dfTags.filter("tag like 's%'").show(10)

In [20]:
spark.sql("""select * from so_tags where tag like 's%'""".stripMargin).show(5)

+---+-------------+
| id|          tag|
+---+-------------+
| 25|      sockets|
| 36|          sql|
| 36|   sql-server|
| 40| structuremap|
| 48|submit-button|
+---+-------------+
only showing top 5 rows



### SQL where with and clause

In [21]:
// dfTags.filter("tag like 's%'").filter("id == 25 or id == 108").show(10)

In [22]:
spark.sql("""select * from so_tags where tag like 's%' and (id = 25 or id = 108)""".stripMargin).show(5)

+---+-------+
| id|    tag|
+---+-------+
| 25|sockets|
|108|    svn|
+---+-------+



### SQL IN clause

In [25]:
// dfTags.filter("id in (25, 108)").show(10)

In [23]:
spark.sql("""select * from so_tags where id in (25, 108)""".stripMargin).show(5)

+---+---------+
| id|      tag|
+---+---------+
| 25|      c++|
| 25|        c|
| 25|  sockets|
| 25|mainframe|
| 25|      zos|
+---+---------+
only showing top 5 rows



### SQL Group By

In [26]:
// dfTags.groupBy("tag").count().show(10)

In [24]:
spark.sql("""select tag, count(*) as count from so_tags group by tag""".stripMargin).show(5)

+-----------+-----+
|        tag|count|
+-----------+-----+
|type-safety|    4|
|    jbutton|    1|
|     iframe|    2|
|  svn-hooks|    2|
|  standards|    7|
+-----------+-----+
only showing top 5 rows



### SQL Group By with having clause

In [27]:
// dfTags.groupBy("tag").count().filter("count > 5").show(10)

In [28]:
spark.sql("""select tag, count(*) as count from so_tags group by tag having count > 5""".stripMargin).show(5)

+-------------+-----+
|          tag|count|
+-------------+-----+
|    standards|    7|
|     keyboard|    8|
|          rss|   12|
|documentation|   15|
|      session|    6|
+-------------+-----+
only showing top 5 rows



### SQL Order by

In [29]:
// dfTags.groupBy("tag").count().filter("count > 5").orderBy("tag").show(10)

In [30]:
spark.sql("""select tag, count(*) as count from so_tags group by tag having count > 5 order by tag""".stripMargin).show(5)

+--------------+-----+
|           tag|count|
+--------------+-----+
|          .net|  351|
|      .net-2.0|   14|
|      .net-3.5|   30|
|         64bit|    7|
|actionscript-3|   22|
+--------------+-----+
only showing top 5 rows



### Typed dataframe, filter and temp table

In [31]:
val dfQuestionsCSV = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("dateFormat","yyyy-MM-dd HH:mm:ss")
    .csv("..\\Resources\\questions_10K.csv")
    .toDF("id", "creation_date", "closed_date", "deletion_date", "score", "owner_userid", "answer_count")

dfQuestionsCSV: org.apache.spark.sql.DataFrame = [id: int, creation_date: timestamp ... 5 more fields]


#### cast columns to data types

In [32]:
val dfQuestions = dfQuestionsCSV.select(
    dfQuestionsCSV.col("id").cast("integer"),
    dfQuestionsCSV.col("creation_date").cast("timestamp"),
    dfQuestionsCSV.col("closed_date").cast("timestamp"),
    dfQuestionsCSV.col("deletion_date").cast("date"),
    dfQuestionsCSV.col("score").cast("integer"),
    dfQuestionsCSV.col("owner_userid").cast("integer"),
    dfQuestionsCSV.col("answer_count").cast("integer")
  )

dfQuestions: org.apache.spark.sql.DataFrame = [id: int, creation_date: timestamp ... 5 more fields]


#### filter dataframe

In [33]:
val dfQuestionsSubset = dfQuestions.filter("score > 400 and score < 410").toDF()

dfQuestionsSubset: org.apache.spark.sql.DataFrame = [id: int, creation_date: timestamp ... 5 more fields]


#### register temp table

In [34]:
dfQuestionsSubset.createOrReplaceTempView("so_questions")

### SQL Inner Join

In [35]:
// dfQuestionsSubset.join(dfTags, Seq("id"), "inner").show(10)

In [36]:
spark.sql("""select t.*, q.* from so_questions q inner join so_tags t on t.id = q.id""".stripMargin).show(5)

+---+---------+---+-------------------+-------------------+-------------+-----+------------+------------+
| id|      tag| id|      creation_date|        closed_date|deletion_date|score|owner_userid|answer_count|
+---+---------+---+-------------------+-------------------+-------------+-----+------------+------------+
|888|   xdebug|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888| phpstorm|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888|debugging|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888|  eclipse|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888|      php|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
+---+---------+---+-------------------+-------------------+-------------+-----+------------+------------+
only showing top 5 rows



### SQL Left Outer Join

In [37]:
// dfQuestionsSubset.join(dfTags, Seq("id"), "left_outer").show(10)

In [38]:
spark.sql("""select t.*, q.* from so_questions q left outer join so_tags t on t.id = q.id""".stripMargin).show(5)

+---+---------+---+-------------------+-------------------+-------------+-----+------------+------------+
| id|      tag| id|      creation_date|        closed_date|deletion_date|score|owner_userid|answer_count|
+---+---------+---+-------------------+-------------------+-------------+-----+------------+------------+
|888|   xdebug|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888| phpstorm|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888|debugging|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888|  eclipse|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
|888|      php|888|2008-08-04 04:48:21|2016-08-04 14:52:00|         null|  405|         131|          30|
+---+---------+---+-------------------+-------------------+-------------+-----+------------+------------+
only showing top 5 rows



### SQL Right Outer Join

In [39]:
// dfTags.join(dfQuestionsSubset, Seq("id"), "right_outer").show(10)

In [40]:
spark.sql("""select t.*, q.* from so_questions q right outer join so_tags t on t.id = q.id""".stripMargin).show(5)

+---+---------------+----+-------------+-----------+-------------+-----+------------+------------+
| id|            tag|  id|creation_date|closed_date|deletion_date|score|owner_userid|answer_count|
+---+---------------+----+-------------+-----------+-------------+-----+------------+------------+
|  1|           data|null|         null|       null|         null| null|        null|        null|
|  4|             c#|null|         null|       null|         null| null|        null|        null|
|  4|       winforms|null|         null|       null|         null| null|        null|        null|
|  4|type-conversion|null|         null|       null|         null| null|        null|        null|
|  4|        decimal|null|         null|       null|         null| null|        null|        null|
+---+---------------+----+-------------+-----------+-------------+-----+------------+------------+
only showing top 5 rows



### Register `User Defined Function (UDF)` Function to prefix a String with so_ short for StackOverflow

In [41]:
def prefixStackoverflow(s: String): String = s"so_$s"

prefixStackoverflow: (s: String)String


#### Register User Defined Function (UDF)

In [42]:
spark.udf.register("prefix_so", prefixStackoverflow _)

res33: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


#### Use udf prefix_so to augment each tag value with so_

In [43]:
spark.sql("""select id, prefix_so(tag) from so_tags""".stripMargin).show(5)

+---+------------------+
| id|UDF:prefix_so(tag)|
+---+------------------+
|  1|           so_data|
|  4|             so_c#|
|  4|       so_winforms|
|  4|so_type-conversion|
|  4|        so_decimal|
+---+------------------+
only showing top 5 rows



### SQL Distinct

In [44]:
spark.sql("""select distinct tag from so_tags""".stripMargin).show(5)

+-----------+
|        tag|
+-----------+
|type-safety|
|    jbutton|
|     iframe|
|  svn-hooks|
|  standards|
+-----------+
only showing top 5 rows



### Closing Spark Session

In [45]:
spark.stop()